In [1]:
# Import stuff
from os.path import join
from pandas import DataFrame

from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.base import Bunch
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.interfaces.fsl.model import GLM
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.algorithms.confounds import CompCor


# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Set study variables
analysis_home = '/home/camachocm2/Analysis/ChEC/fmri_proc'
raw_dir = analysis_home + '/raw'
preproc_dir = analysis_home + '/preproc'
firstlevel_dir = analysis_home + '/subjectlevel'
secondlevel_dir = analysis_home + '/grouplevel'
workflow_dir = analysis_home + '/workflows'
timing_file = '/home/camachocm2/Analysis/ChEC/misc/affect.txt'
#subject_info = DataFrame.read_csv(analysis_home + '/../misc/subjs.csv')
#subjects_list = subject_info['SubjID'].tolist()
subjects_list = ['pilot002']

# data collection specs
TR = 0.8 #in seconds


In [2]:
# Select subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Pull files
file_template = {'preproc_func': preproc_dir + '/coreg_func/{subjid}/preproc_func.nii.gz', 
                 'motion': preproc_dir + '/displacement/{subjid}/fd.txt', 
                 'outliers': preproc_dir + '/art_outliers/{subjid}/art.preproc_func_outliers.txt'}

selectfiles = Node(SelectFiles(file_template), name='selectfiles')

# Sink data of interest (mostly for QC)
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = firstlevel_dir,
                        container = firstlevel_dir,
                        substitutions = substitutions), 
                name='datasink')

In [3]:
## Timing handling nodes
def affectiveTiming(timing_file, motion, outliers):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from numpy import genfromtxt, zeros, column_stack, savetxt
    from os.path import abspath 
    
    motion = genfromtxt(motion, delimiter=None, dtype=None, skip_header=0)
    timing = genfromtxt(timing_file, delimiter=None, dtype=None, skip_header=0)
    censor_vol_list = genfromtxt(outliers, delimiter=None, dtype=None, skip_header=0)
    
    try:
        c = censor_vol_list.size
    except:
        c = 0
    
    d=len(motion)

    if c > 1:
        scrubbing = zeros((d,c),dtype=int)
        for t in range(c):
            scrubbing[censor_vol_list[t],t] = 1
        regres_matrix = column_stack((timing, motion, scrubbing))
    elif c == 1:
        scrubbing = zeros((d,c),dtype=int)
        scrubbing[censor_vol_list] = 1
        regres_matrix = column_stack((timing, motion, scrubbing))
    else:
        regres_matrix = column_stack((timing, motion))
    
    design_file_name = 'matrix.txt'
    savetxt(design_file_name, regres_matrix)
    design_file = abspath(design_file_name)
    
    return(design_file)

In [6]:
affective_timing = Node(Function(input_names=['timing_file', 'motion', 'outliers'],
                                 output_names=['design_file'],
                                 function=affectiveTiming), 
                        name='affective_timing')
affective_timing.inputs.timing_file = timing_file

#high pass filter
high_pass = Node(TemporalFilter(highpass_sigma=128/TR), 
                 name='high_pass')

#run the GLM
estmodel = Node(GLM(dat_norm = True,
                    out_file = 'betas.nii.gz', 
                    out_cope='cope.nii.gz',
                    out_t_name = 'tstat.nii.gz'), 
                name= 'estmodel')

In [7]:
L1workflow = Workflow(name='L1workflow')
L1workflow.connect([(infosource,selectfiles,[('subjid','subjid')]),
                    (selectfiles, affective_timing,[('motion','motion')]),
                    (selectfiles, affective_timing,[('outliers','outliers')]),
                    (affective_timing,estmodel,[('design_file','design')]),
                    (selectfiles,estmodel,[('preproc_func','in_file')]),
                    
                    (affective_timing, datasink, [('design_file','design_tnp')]),
                    (estmodel, datasink, [('out_cope','copes')]),
                    (estmodel, datasink, [('out_t','tstats')]),
                    (estmodel, datasink, [('out_file','betas')])
                   ])
L1workflow.base_dir = join(workflow_dir)
L1workflow.write_graph(graph2use='flat')
L1workflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

180510-14:46:09,242 workflow INFO:
	 Generated workflow graph: /home/camachocm2/Analysis/ChEC/fmri_proc/workflows/L1workflow/graph.png (graph2use=flat, simple_form=True).
180510-14:46:09,273 workflow INFO:
	 Workflow L1workflow settings: ['check', 'execution', 'logging', 'monitoring']
180510-14:46:09,288 workflow INFO:
	 Running in parallel.
180510-14:46:09,292 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 4/4.
180510-14:46:09,366 workflow INFO:
	 [Node] Setting-up "L1workflow.selectfiles" in "/home/camachocm2/Analysis/ChEC/fmri_proc/workflows/L1workflow/_subjid_pilot002/selectfiles".
180510-14:46:09,378 workflow INFO:
	 [Node] Running "selectfiles" ("nipype.interfaces.io.SelectFiles")
180510-14:46:09,386 workflow INFO:
	 [Node] Finished "L1workflow.selectfiles".
180510-14:46:11,294 workflow INFO:
	 [Job 0] Completed (L1workflow.selectfiles).
180510-14:46:11,299 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs re

Process NonDaemonPoolWorker-11:
Process NonDaemonPoolWorker-8:
Process NonDaemonPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    util._exit_function()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiproce

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-13ec03f60dd1>", line 15, in <module>
    L1workflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})
  File "/usr/local/lib/python3.5/dist-packages/nipype/pipeline/engine/workflows.py", line 595, in run
    runner.run(execgraph, updatehash=updatehash, config=self.config)
  File "/usr/local/lib/python3.5/dist-packages/nipype/pipeline/plugins/base.py", line 195, in run
    sleep(max(0, sleep_til - time()))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During ha

KeyboardInterrupt: 

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
